# Nishant Andoriya
# 201951105
# Assigment 12

In [5]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [24]:
location_cookie_cats_data = "cookie_cats.csv"

In [6]:
cookie_cats_data = pd.read_csv(location_cookie_cats_data);
cookie_cats_data

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True
...,...,...,...,...,...
90184,9999441,gate_40,97,True,False
90185,9999479,gate_40,30,False,False
90186,9999710,gate_30,28,True,False
90187,9999768,gate_40,51,True,False


In [7]:
temp_data = cookie_cats_data.set_index("version")

In [8]:
version_data = temp_data.loc["gate_30"].reset_index()
version_data["gate_40"] = temp_data.loc["gate_40"].reset_index()["sum_gamerounds"]
version_data["gate_30"] = temp_data.loc["gate_30"].reset_index()["sum_gamerounds"]
version_data = version_data.iloc[:,5:]
version_data

,gate_40,gate_30
0,165,3
1,1,38
2,179,0
3,187,0
4,2,39
...,...,...
44695,43,14
44696,9,7
44697,0,21
44698,5,10


In [9]:
# Derivation using Observed means method 
observed_variance_data = cookie_cats_data.groupby('version').mean().var()[0]
print('mean observed:', cookie_cats_data.groupby('version').mean().values.ravel())
print('variance:', observed_variance_data)

mean observed: [4.98756372e+06 5.24562640e+01 4.48187919e-01 1.90201342e-01
 5.00907258e+06 5.12987755e+01 4.42282750e-01 1.82000044e-01]
variance: 231315446.31845072


In [10]:
def perm_test(data):
    data = data.copy()
    data['sum_gamerounds'] = np.random.permutation(data['sum_gamerounds'].values)
    return data.groupby('sum_gamerounds').mean().var()[0] 
permutation_variance = [perm_test(cookie_cats_data) for _ in range(3000)]

In [11]:
print('Pr(Prob) is: ', np.mean([var > observed_variance_data for var in permutation_variance]))

Pr(Prob) is:  1.0


In [12]:
#  Using stats.f.oneway
f_value, p_value = stats.f_oneway(version_data['gate_30'], version_data['gate_40'])
print("P value using stats.f.oneway: ",p_value)
print("F value using stats.f.oneway: ",f_value)

P value using stats.f.oneway:  0.3947911230235258
F value using stats.f.oneway:  0.724141283273616


In [13]:
#  Derivation using F-stastic
mu = version_data.mean()
group_mu = mu.mean()
print("Group wise mean of data: ", mu.values)
print("Grand mean of data: ", group_mu)

Group wise mean of data:  [51.34217002 52.45626398]
Grand mean of data:  51.89921700223714


In [14]:
degree_freedom = 89398
group_degree_freedom = 1 

In [15]:
# Sum of Squares (SS) of error, group wise
temp = version_data - mu
temp = temp**2
temp = temp.sum().sum()
SS = 1 * temp
temp

3424724671.0055933

In [16]:
# # Mean of Sum of squares of error group wise
MSS=SS / degree_freedom 
MSS

38308.7392447884

In [17]:
# # Sum of squares of error for whole data
GSS = ((mu - group_mu) ** 2).sum() * 2
temp = (mu - group_mu) ** 2
GSS

1.24120535111032

In [18]:
# Mean of Sum of squares of error group wise
GMSS = GSS / group_degree_freedom 
GMSS

1.24120535111032

In [19]:
# F value
F_value = GMSS / MSS
F_value

3.2400057417164313e-05

In [20]:
def perm_test(df):
    df = df.copy()
    df['sum_gamerounds'] = np.random.permutation(df['sum_gamerounds'].values)
    tmp=df.set_index("version")
    df2=tmp.loc["gate_30"].reset_index()
    df2["gate_40"]=tmp.loc["gate_40"].reset_index()["sum_gamerounds"]
    df2["gate_30"]=tmp.loc["gate_30"].reset_index()["sum_gamerounds"]
    df2=version_data.iloc[:,5:]
    mu=df2.mean()
    SS=((df2-mu)**2).sum().sum()
    MSS=SS/degree_freedom
    GSS=((mu - group_mu)**2).sum()*2
    GMSS=GSS / group_degree_freedom 
    F_value_perm = GMSS/MSS
    return F_value_perm
permutation_variance = [perm_test(cookie_cats_data) for _ in range(3000)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  


In [21]:
print('Pr(Prob) is: ', np.mean([fv > F_value for fv in permutation_variance]))

Pr(Prob) is:  0.0


In [22]:
# Derivation using ANOVA table
model = ols("sum_gamerounds ~ version", data=cookie_cats_data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
version,3.020603e+04,1.0,0.793957,0.372909
Residual,3.431158e+09,90187.0,NaN,NaN


#OBSERVATION AND RESULTS


Since the p-Value of each of the derived techniques is higher than the alpha (i.e. 0.05) (some techniques cannot be derived), we should not reject the null hypothesis. 